In [1]:
python_code = '''
import random
import sys
# input_val = '0,1,2,3'
input_val = sys.argv[1]
exit_code = int(random.choice(input_val.split(",")))
print(exit_code)
sys.exit(exit_code)
'''

In [2]:
import kfp
from kfp import dsl

@dsl.pipeline(name='Retry random failures', description='A pipeline with retry.')
def retry_pipeline():
    random_exit_op('0,1,2,3').set_retry(10)

def random_exit_op(numbers_str):
    return dsl.ContainerOp(
        name='random_failure',
        image='python:alpine3.6',
        command=['python', '-c'],
        arguments=[python_code, numbers_str]
    )

In [3]:
pipelineGzFile = 'hahahoho.zip'

kfp.compiler.Compiler().compile(
    retry_pipeline, 
    pipelineGzFile
)

client = kfp.Client()
my_exp = client.create_experiment(
    name='Basic Experiment'
)
my_run = client.run_pipeline(
    my_exp.id, 
    'Retry pipeline',
    pipelineGzFile
)